In [6]:
# Import library yang dibutuhkan
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
import warnings

# Mengabaikan peringatan yang mungkin muncul untuk tampilan yang lebih bersih
warnings.filterwarnings('ignore')

class StrokeDataPreprocessor:
    """
    Sebuah kelas untuk melakukan semua langkah prapemrosesan pada dataset stroke
    untuk persiapan pembuatan Sistem Fuzzy.
    """
    def __init__(self):
        """Inisialisasi imputer dan encoder"""
        self.imputer_bmi = SimpleImputer(strategy='median')
        self.label_encoders = {}
        self.processed_data = None

    def _load_data(self, file_path):
        """Memuat data dari file CSV."""
        print("1. Memuat dataset...")
        try:
            return pd.read_csv(file_path)
        except FileNotFoundError:
            print(f"Error: File tidak ditemukan di {file_path}")
            return None

    def _handle_anomalies_and_irrelevant_columns(self, df):
        """Menghapus kolom tidak relevan dan anomali data."""
        print("2. Menghapus kolom 'id' dan baris anomali (gender='Other')...")
        
        # Hapus kolom 'id' karena tidak relevan
        if 'id' in df.columns:
            df = df.drop('id', axis=1)
        
        # Hapus baris dengan gender 'Other' karena hanya ada 1 data
        if 'gender' in df.columns:
            df = df[df['gender'] != 'Other']
            
        return df

    def _handle_missing_values(self, df):
        """Menangani nilai yang hilang pada kolom 'bmi'."""
        print("3. Menangani missing values pada kolom 'bmi' dengan median...")
        df['bmi'] = self.imputer_bmi.fit_transform(df[['bmi']])
        return df

    def _encode_categorical_features(self, df):
        """Melakukan label encoding pada fitur kategorikal."""
        print("4. Melakukan encoding pada fitur-fitur kategorikal...")
        categorical_cols = ['gender', 'ever_married', 'work_type', 'Residence_type', 'smoking_status']
        for col in categorical_cols:
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
            self.label_encoders[col] = le
        return df

    def process(self, file_path):
        """
        Menjalankan seluruh pipeline prapemrosesan data.
        """
        # Langkah 1: Muat data
        data = self._load_data(file_path)
        if data is None:
            return None
        
        # Langkah 2: Handle kolom yang tidak relevan
        data = self._handle_anomalies_and_irrelevant_columns(data)
        
        # Menghapus duplikasi data
        data = data.drop_duplicates().reset_index(drop=True)
        
        # Langkah 3: Handle missing values
        data = self._handle_missing_values(data)
        
        # Langkah 4: Encoding fitur kategorikal
        data = self._encode_categorical_features(data)
        print("\n=== Proses Prapemrosesan Selesai! ===")
        self.processed_data = data
        return data

    def explore_processed_data(self):
        """Menampilkan informasi ringkas mengenai data yang sudah diproses."""
        if self.processed_data is not None:
            print("\n=== INFORMASI DATASET SETELAH DIPROSES ===")
            print(f"Bentuk Data: {self.processed_data.shape}")
            print("\nInfo Tipe Data:")
            self.processed_data.info()
            print("\nContoh 5 Baris Data Teratas:")
            print(self.processed_data.head())
            print("\nStatistik Deskriptif Fitur Numerik:")
            print(self.processed_data[['age', 'avg_glucose_level', 'bmi']].describe())
        else:
            print("Data belum diproses. Jalankan method process() terlebih dahulu.")
            
    def save_processed_data(self, output_path):
        """Menyimpan data yang sudah bersih ke file CSV."""
        if self.processed_data is not None:
            self.processed_data.to_csv(output_path, index=False)
            print(f"\nData yang sudah diproses telah disimpan di: {output_path}")
        else:
            print("Tidak ada data untuk disimpan. Jalankan method process() terlebih dahulu.")

In [7]:
preprocessor = StrokeDataPreprocessor()

In [8]:
# 2. Jalankan seluruh proses
clean_df = preprocessor.process('healthcare-dataset-stroke-data.csv')

1. Memuat dataset...
2. Menghapus kolom 'id' dan baris anomali (gender='Other')...
3. Menangani missing values pada kolom 'bmi' dengan median...
4. Melakukan encoding pada fitur-fitur kategorikal...

=== Proses Prapemrosesan Selesai! ===


In [5]:
if clean_df is not None:
    preprocessor.explore_processed_data()
    
    # 4. Simpan data bersih ke file baru untuk digunakan di tahap selanjutnya
    preprocessor.save_processed_data('stroke_preprocessed_for_fuzzy.csv')


=== INFORMASI DATASET SETELAH DIPROSES ===
Bentuk Data: (5109, 11)

Info Tipe Data:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5109 entries, 0 to 5108
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   gender             5109 non-null   int32  
 1   age                5109 non-null   float64
 2   hypertension       5109 non-null   int64  
 3   heart_disease      5109 non-null   int64  
 4   ever_married       5109 non-null   int32  
 5   work_type          5109 non-null   int32  
 6   Residence_type     5109 non-null   int32  
 7   avg_glucose_level  5109 non-null   float64
 8   bmi                5109 non-null   float64
 9   smoking_status     5109 non-null   int32  
 10  stroke             5109 non-null   int64  
dtypes: float64(3), int32(5), int64(3)
memory usage: 339.4 KB

Contoh 5 Baris Data Teratas:
   gender   age  hypertension  heart_disease  ever_married  work_type  \
0       1  67.0    